[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/normalization)

# Select Your IPTS 

<img src='__docs/__all/shift_enter.gif' />

In [1]:
from __code.normalization_new_ui import *

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_roi_selection.ui')
from __code.roi_selection_ui import Interface

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_roi_selection.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_roi_selection.py


** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19213', 'IPTS-19558', 'IPTS-19621', 'IPTS-19626', 'IPTS-19799', 'IPTS-20209-TOPAZ', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Python Import

In [2]:
%gui qt

# Loading Images 

In [3]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

button_layout Layout(border='1px solid gray', margin='5px 40px')
ok_layout Layout(border='1px solid blue', margin='auto 40px 5px')


Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='warning', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-19213', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                |     Mon Mar  5 10:48:15 2018', ' Sample 1A_normalized     |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

...

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Dark Field files (optional) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Button(button_style='success', description='<< Previous Step', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to previous step'), Label(value='         ', layout=Layout(width='70%')), Button(button_style='success', description='Next Step>>', layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-19213', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), SelectMultiple(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                |     Mon Mar  5 10:48:15 2018', ' Sample 1A_normalized     |     Directory'), value=())), layout=Layout(width='80%')), VBox(children=(Button(description='Change directory', layout=Layout(border='1px solid gray', margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(border='1px solid blue', margin='auto 40px 5px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px')))), HTML(value='')))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=5), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

HBox(children=(Label(value='Loading ob', layout=Layout(width='20%')), IntProgress(value=0, max=2), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

# Select Background Region 

In [4]:
o_norm = sample_panel.o_norm_handler
o_gui = Interface(o_norm=o_norm.o_norm)
o_gui.show()

Leaving Parameters Selection UI


# Recap. 

In [5]:
o_gui.recap()

Type,Number,Size (height*width)
sample,5,2048*2048
ob,2,2048*2048
df,0,N/A*N/A


# Normalization

In [6]:
o_norm.run_normalization(dict_roi=o_gui.roi_selected)

HBox(children=(Label(value='Normalization', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

# Export 

In [7]:
o_norm.select_export_folder()

HBox(children=(Button(button_style='success', description='Jump to IPTS-19213 Shared Folder', layout=Layout(border='1px solid gray', width='30%'), style=ButtonStyle()), Button(button_style='success', description='Jump to My Home Folder', layout=Layout(border='1px solid gray', width='30%'), style=ButtonStyle())))


button_layout Layout(border='1px solid gray', margin='5px 40px')
ok_layout Layout(border='1px solid blue', margin='auto 40px 5px')


VBox(children=(Label(value='Select Output Folder', layout=Layout(width='250px')), VBox(children=(HBox(children=(HBox(children=(Text(value='/Users/j35/git/IPTS/python_notebooks', description='Location: ', layout=Layout(width='100%'), placeholder=''), Button(description='Jump', layout=Layout(border='1px solid gray', margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px', width='100%')),)), HBox(children=(VBox(children=(HBox(children=(Label(value='Search:'), Text(value='', layout=Layout(width='50%'))), layout=Layout(justify_content='flex-end')), Select(description='Select', layout=Layout(height='260px', width='99%'), options=(' .', ' ..', ' .DS_Store                                                 |     Fri Mar  2 13:10:38 2018', ' .calculate_water_intake_profile.ipynb                     |     Mon Jan 22 12:21:09 2018', ' .git                                                      |     Directory', ' .gitignore                                                |     Mon Jan 22 12:21:09 2018', ' .idea                                                     |     Directory', ' .ipynb_checkpoints                                        |     Directory', ' .normalization.ipynb                                      |     Mon Jan 22 12:21:09 2018', ' .normalization_by_range_of_images_NO_GUIDE.ipynb          |     Mon Jan 22 12:21:09 2018', ' .pre_commit.py                                            |     Mon Jan 22 12:21:09 2018', ' .profile_vs_file_index_PROTOTYPE.ipynb                    |     Mon Jan 22 12:21:09 2018', ' README.md                                                 |     Mon Jan 22 12:21:09 2018', ' TOPAZ_config_generator.ipynb                              |     Tue Mar  6 12:49:11 2018', ' Tutorials                                                 |     Directory', ' __code                                                    |     Directory', ' __docs                                                    |     Directory', ' bin_images.ipynb                                          |     Mon Mar  5 17:29:36 2018', ' calculate_water_intake_profile.ipynb                      |     Mon Mar  5 17:29:36 2018', ' calculate_water_intake_profile_OLD.ipynb                  |     Mon Mar  5 17:29:37 2018', ' combine_folders.ipynb                                     |     Mon Mar  5 17:29:37 2018', ' combine_images.ipynb                                      |     Mon Mar  5 17:29:37 2018', ' create_list_of_file_name_vs_time_stamp.ipynb              |     Mon Mar  5 17:29:37 2018', ' deal_files.ipynb                                          |     Mon Mar  5 17:29:38 2018', ' display_and_export_images_with_metadata_profile.ipynb     |     Mon Mar  5 17:29:38 2018', ' display_and_export_images_with_timestamp.ipynb            |     Mon Mar  5 17:29:38 2018', ' display_counts_of_region_vs_stack.ipynb                   |     Mon Mar  5 17:29:38 2018', ' display_integrated_stack_of_images.ipynb                  |     Mon Mar  5 17:29:38 2018', ' figuring_out_how_to_control_size_of_pyqtgraph.ipynb       |     Mon Mar  5 17:29:39 2018', ' file_name_and_metadata_vs_time_stamp.ipynb                |     Mon Mar  5 17:29:39 2018', ' fix_images.ipynb                                          |     Mon Mar  5 17:29:39 2018', ' fix_images_with_negative_pixels.ipynb                     |     Mon Mar  5 17:29:39 2018', ' format_filename_index.ipynb                               |     Mon Mar  5 17:29:40 2018', ' from_dsc_time_info_to_ascii_file_vs_time.ipynb            |     Mon Mar  5 17:29:40 2018', ' install_me.sh                                             |     Mon Jan 22 12:21:09 2018', ' ipts_18813.ipynb                                          |     Tue Mar  6 09:59:46 2018', ' linear_profile.ipynb                                      |     Mon Mar  5 17:29:40 2018', ' list_tiff_metadata.ipynb                                  |     Mon Mar  5 17:29:41 2018', ' locate_p

/SNS/users/j35


FileNotFoundError: [Errno 2] No such file or directory: '/SNS/users/j35'

/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-19213/shared


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-19213/shared'

In [ ]:
o_norm.export()